# Load videos, synch frames with command velocities

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os   
# Define directories
video_dir = r'C:\Users\anton\Documents\Competition Project\Videos+timestamps'
commands_dir = r'C:\Users\anton\Documents\Competition Project\commands'

# Get a list of all .mp4 video files and corresponding timestamp CSVs
video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]
timestamp_files = {f: os.path.join(video_dir, f) for f in os.listdir(video_dir) if f.startswith('timestamps_') and f.endswith('.csv')}

# Frame preprocessing parameters
img_size = (112, 112)  # Resize dimensions
max_sync_gap = 0.5  # Maximum time gap for velocity synchronization (in seconds)

# Initialize a dictionary to store data for each video
data = {}

# Loop through each video
for video_file in video_files:
    video_path = os.path.join(video_dir, video_file)
    print(f"Processing video: {video_file}")

    # Find the corresponding timestamp CSV
    base_name = os.path.splitext(video_file)[0].split('_')[-1]
    timestamp_file = next((path for name, path in timestamp_files.items() if base_name in name), None)

    if not timestamp_file:
        print(f"No timestamp file found for video: {video_file}. Skipping.")
        continue

    # Load frame timestamps from the timestamp CSV
    try:
        frame_timestamps = pd.read_csv(timestamp_file)

        # Ensure required columns exist
        if 'frame_number' not in frame_timestamps.columns or 'timestamp' not in frame_timestamps.columns:
            print(f"Invalid timestamp file format for {video_file}. Skipping.")
            continue

        # Validate timestamp column
        if not np.issubdtype(frame_timestamps['timestamp'].dtype, np.number):
            print(f"Timestamp column in {video_file} is not numeric. Converting.")
            frame_timestamps['timestamp'] = pd.to_numeric(frame_timestamps['timestamp'], errors='coerce')
    except Exception as e:
        print(f"Error reading timestamp file for {video_file}: {e}")
        continue

    # Open the corresponding commands CSV
    timestamp = "_".join(os.path.splitext(video_file)[0].split('_')[-2:])
    commands_file = os.path.join(commands_dir, f"commands_{timestamp}.csv")

    if not os.path.exists(commands_file):
        print(f"No commands file found for video: {video_file}. Skipping.")
        continue

    # Load commands
    try:
        commands = pd.read_csv(commands_file)

        # Ensure required columns exist
        if 'timestamp' not in commands.columns or 'linear_velocity' not in commands.columns or 'angular_velocity' not in commands.columns:
            print(f"Invalid commands file format for {video_file}. Skipping.")
            continue

        # Validate timestamp and velocity columns
        commands['timestamp'] = pd.to_numeric(commands['timestamp'], errors='coerce')
        commands['linear_velocity'] = pd.to_numeric(commands['linear_velocity'], errors='coerce')
        commands['angular_velocity'] = pd.to_numeric(commands['angular_velocity'], errors='coerce')
    except Exception as e:
        print(f"Error reading commands file for {video_file}: {e}")
        continue

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Failed to open video: {video_file}")
        continue

    # Process frames
    frames = []
    frame_data = []

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # End of video

        # Preprocess the frame
        frame_resized = cv2.resize(frame, img_size, interpolation=cv2.INTER_AREA) / 255.0

        # Get the exact timestamp for this frame
        frame_timestamp = frame_timestamps.loc[frame_timestamps['frame_number'] == frame_count, 'timestamp']

        if not frame_timestamp.empty:
            frames.append(frame_resized)
            frame_data.append({
                'timestamp': frame_timestamp.values[0],
                'frame_index': frame_count
            })

        frame_count += 1

    cap.release()
    print(f"Extracted {len(frames)} frames from video: {video_file}")

    # Convert frame data to DataFrame
    frame_df = pd.DataFrame(frame_data)

    # Perform time-based join with commands
    aligned_data = pd.merge_asof(
        frame_df,
        commands.sort_values('timestamp'),
        on='timestamp',
        direction='backward',
        tolerance=max_sync_gap  # Max time difference allowed
    )

    # Handle cases where no recent command was found
    if aligned_data['linear_velocity'].isna().any():
        print(f"Warning: Some frames in {video_file} could not be synchronized with commands.")

        # Optional: forward fill missing velocities
        aligned_data['linear_velocity'].ffill()
        aligned_data['angular_velocity'].ffill()
    # Remove any remaining rows with NaN velocities
    aligned_data.dropna(subset=['linear_velocity', 'angular_velocity'], inplace=True)

    # Create final data structure
    synchronized_frames = [frames[int(idx)] for idx in aligned_data['frame_index']]
    synchronized_commands = aligned_data[['timestamp', 'linear_velocity', 'angular_velocity']].values

    # Store results for this video
    data[video_file] = {
        "frames": np.array(synchronized_frames),
        "commands": synchronized_commands,
    }

    print(f"Synchronized {len(synchronized_frames)} frames with commands for {video_file}")

print("Finished processing all videos.")

# Check if the data dictionary contains any processed videos
if not data:
    print("No videos were successfully processed. Please check your data and paths.")
else:
    # Example: Access frames and commands for a specific video
    example_video = next(iter(data.keys()))  # Get the first successfully processed video
    frames = data[example_video]["frames"]
    commands = data[example_video]["commands"]
    print(f"Frames shape: {frames.shape}")
    print(f"Commands shape: {commands.shape}")

## Verify video data is correctly encoded

In [ ]:
import matplotlib.pyplot as plt

# Retrieve the aligned data for the example video
aligned_timestamps = data[example_video]["commands"]  # Commands and timestamps
frames = data[example_video]["frames"]

# Visualize the first 200 frames with additional information
for i in range(100,102):  # Ensure we don't exceed the number of frames
    # Retrieve timestamp and command for the current frame
    frame_timestamp = aligned_timestamps[i][0] if i < len(aligned_timestamps) else None
    linear_velocity = aligned_timestamps[i][1] if i < len(aligned_timestamps) else None
    angular_velocity = aligned_timestamps[i][2] if i < len(aligned_timestamps) else None

    # Display the frame
    plt.imshow(frames[i])
    title_text = f"Frame {i} | Timestamp: {frame_timestamp:.3f} | Linear Vel: {linear_velocity:.3f} | Angular Vel: {angular_velocity:.3f}"
    plt.title(title_text)
    plt.axis('off')
    plt.show()


#Divide data for model training, verification and testing

In [3]:
import os
import random
import pickle

# Define the directory to save preprocessed data
output_dir = r'C:\Users\anton\Documents\Competition Project\Preprocessed Data'

# Randomly shuffle video keys
video_keys = list(data.keys())
random.shuffle(video_keys)

# Define split ratios
train_split = int(0.7 * len(video_keys))
val_split = int(0.9 * len(video_keys))

# Split data
train_videos = video_keys[:train_split]
val_videos = video_keys[train_split:val_split]
test_videos = video_keys[val_split:]

# Save split datasets
split_dir = os.path.join(output_dir, "splits")
os.makedirs(split_dir, exist_ok=True)


In [4]:
#make pickle
for split_name, split_videos in [("train", train_videos), ("val", val_videos), ("test", test_videos)]:
    split_file = os.path.join(split_dir, f"{split_name}_split.pkl")
    with open(split_file, "wb") as f:
        split_data = {video: data[video] for video in split_videos}
        pickle.dump(split_data, f)
    print(f"Saved {split_name} split with {len(split_videos)} videos to {split_file}")

Saved train split with 4 videos to C:\Users\anton\Documents\Competition Project\Preprocessed Data\splits\train_split.pkl
Saved val split with 2 videos to C:\Users\anton\Documents\Competition Project\Preprocessed Data\splits\val_split.pkl
Saved test split with 1 videos to C:\Users\anton\Documents\Competition Project\Preprocessed Data\splits\test_split.pkl


##**start running code cells here if you have no new data to add for training**

In [2]:
import os

# split_dir = "/content/drive/My Drive/BASC, third year/ENPH 353/Competition Project/Preprocessed Data/splits"
split_dir = r'C:\Users\anton\Documents\Competition Project\Preprocessed Data'

train_path = os.path.join(split_dir, "train_split.pkl")
val_path = os.path.join(split_dir, "val_split.pkl")
test_path = os.path.join(split_dir, "test_split.pkl")

## VERIFY

In [3]:
# Check if split files exist
# split_dir = "/content/drive/My Drive/BASC, third year/ENPH 353/Competition Project/Preprocessed Data/splits"
split_dir = r'C:\Users\anton\Documents\Competition Project\Preprocessed Data\splits'
split_files = ["train_split.pkl", "val_split.pkl", "test_split.pkl"]

for split_file in split_files:
    split_path = os.path.join(split_dir, split_file)
    if os.path.exists(split_path):
        print(f"File exists: {split_path}")
    else:
        print(f"Missing file: {split_path}")

File exists: C:\Users\anton\Documents\Competition Project\Preprocessed Data\splits\train_split.pkl
File exists: C:\Users\anton\Documents\Competition Project\Preprocessed Data\splits\val_split.pkl
File exists: C:\Users\anton\Documents\Competition Project\Preprocessed Data\splits\test_split.pkl


In [4]:
import pickle

# Load and inspect splits
for split_file in split_files:
    split_path = os.path.join(split_dir, split_file)
    with open(split_path, "rb") as f:
        split_data = pickle.load(f)

    print(f"{split_file}: {len(split_data)} videos")
    for video_key, video_data in list(split_data.items())[:2]:  # Display the first 2 entries
        print(f"  Video: {video_key}")
        print(f"    Frames shape: {video_data['frames'].shape}")
        print(f"    Commands shape: {video_data['commands'].shape}")


train_split.pkl: 4 videos
  Video: video_output_20241130_003337.mp4
    Frames shape: (2005, 112, 112, 3)
    Commands shape: (2005, 3)
  Video: video_output_20241130_003756.mp4
    Frames shape: (2245, 112, 112, 3)
    Commands shape: (2245, 3)
val_split.pkl: 2 videos
  Video: video_output_20241130_004101.mp4
    Frames shape: (1560, 112, 112, 3)
    Commands shape: (1560, 3)
  Video: video_output_20241130_004447.mp4
    Frames shape: (1847, 112, 112, 3)
    Commands shape: (1847, 3)
test_split.pkl: 1 videos
  Video: video_output_20241130_002438.mp4
    Frames shape: (2107, 112, 112, 3)
    Commands shape: (2107, 3)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Load train split
train_split_path = os.path.join(split_dir, "train_split.pkl")
with open(train_split_path, "rb") as f:
    train_data = pickle.load(f)

# Select a sample video
sample_video = next(iter(train_data.keys()))
sample_data = train_data[sample_video]

# Inspect sample frames and commands
frames = sample_data["frames"]
commands = sample_data["commands"]

print(f"Sample video: {sample_video}")
print(f"Frames shape: {frames.shape}")
print(f"Commands shape: {commands.shape}")

# Visualize a few frames with commands
for i in range(min(2, len(frames))):
    plt.imshow(frames[i])
    command_text = f"Linear Vel: {commands[i, 1]:.2f}, Angular Vel: {commands[i, 2]:.2f}"
    plt.title(f"Frame {i} | {command_text}")
    plt.axis("off")
    plt.show()


#INSTANTIATE NEURAL NETWORK

##ensure right version of tensorflow is installed

In [6]:
import tensorflow as tf
print(tf.__version__)  # Should output 2.13.1


2.13.1


In [7]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


##Load data for model use

In [8]:
import tensorflow as tf
import os

def data_generator(data, batch_size=32):
    keys = list(data.keys())
    num_samples = sum(len(video_data["frames"]) for video_data in data.values())
    while True:
        X, y = [], []
        for video_key in keys:
            video_data = data[video_key]
            frames = video_data["frames"]
            commands = video_data["commands"]
            for i in range(len(frames)):
                X.append(frames[i])
                y.append(commands[i, 1:])  # Skip timestamp
                if len(X) == batch_size:
                    yield np.array(X), np.array(y)
                    X, y = [], []
        if X:  # Yield remaining samples in the last batch
            yield np.array(X), np.array(y)

In [9]:
import pickle

# Load preprocessed splits
#split_dir = "/content/drive/My Drive/BASC, third year/ENPH 353/Competition Project/Preprocessed Data/splits"
split_dir = r'C:\Users\anton\Documents\Competition Project\Preprocessed Data\splits'


train_path = os.path.join(split_dir, "train_split.pkl")
val_path = os.path.join(split_dir, "val_split.pkl")

with open(train_path, "rb") as f:
    train_data = pickle.load(f)

with open(val_path, "rb") as f:
    val_data = pickle.load(f)

# Create generators for training and validation data
train_generator = data_generator(train_data, batch_size=32)
val_generator = data_generator(val_data, batch_size=32)


In [10]:
import numpy as np

# Test train generator
X_batch, y_batch = next(train_generator)
print(f"Batch shape: {X_batch.shape}, {y_batch.shape}")

Batch shape: (32, 112, 112, 3), (32, 2)


In [14]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 0


##define model

In [11]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(112, 112, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(2, activation='linear')  # Output: linear_velocity, angular_velocity
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 110, 110, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 55, 55, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 53, 53, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 26, 26, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 12, 12, 128)       0

##train the model

In [ ]:
# Calculate steps per epoch
train_steps = sum(len(video_data["frames"]) for video_data in train_data.values()) // 32
val_steps = sum(len(video_data["frames"]) for video_data in val_data.values()) // 32

# Define callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint(
    filepath=r'C:\Users\anton\Documents\Competition Project\Trained Models\imitation_model_3.h5',
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1,
    save_format='h5'  # Explicitly set save format to HDF5
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Train model
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    validation_data=val_generator,
    validation_steps=val_steps,
    epochs=20,
    verbose=1,
    callbacks=[checkpoint, early_stopping]
)

# Visualize training progress
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3504893209.py, line 9)

##save model

In [14]:
model.save(r'C:\Users\anton\Documents\Competition Project\Trained Models\imitation_model_3.h5')

C:\Users\anton\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
import tensorflow as tf

model_path = "/content/drive/My Drive/Competition Project/Trained Models/best_model.h5"
try:
    model = tf.keras.models.load_model(model_path)
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")

Model loaded successfully!


In [19]:
import tensorflow as tf
print(tf.__version__)


2.13.1
